# Project Description

The task of this project is to develop an application for product configuration using the ASP system clinguin.

## Product configuration

The goal of product configuration, as it name says, is to configure a product.
In our running example we consider the configuration of a new bike.

The input to product configuration contains a description of the possible configurations of a product.
This is done in two parts:
1. The first part describes the possible components of a product and their attributes.
For example, a bike must have a front wheel and a rear wheel of some size, and it may have a bag with some number of pockets.
2. The second part describes some constraints over the components.
For example, one constraint can be that the size of the wheels of a bike must be the same.

The input may also contain a selection of components of the product and some of their attributes.
This selection may be complete or incomplete.
For example, the input may contain a complete selection of the components of a bike,
or just consist of a selection of wheels.

The task of product configuration is to
find a possible configuration of the product that is consistent with the given selection.
If there is no such possible configuration,
then the output should explain why there is no possible configuration.

## Clinguin

Clinguin is an extension of clingo to create interactive User Interfaces (UIs) using ASP.

UIs are defined as sets of atoms, which are then rendered by a frontend, and 
provide continuous interaction with an ASP solver based on user-triggered events.

The documentation about clinguin is available here:
* https://clinguin.readthedocs.io

# Resources

## COOM Language

COOM is a language for product configuration. 
It covers the first part mentioned above: 
1. The description of the components and their attributes.
2. The specification of constraints over them.

The documentation of COOM is available at [doc/coom.ipynb](doc/coom.ipynb).

A basic bike can be described as follows:

In [1]:
! cat coom/instances/bike-1-basic.coom

product {
	Wheel	frontWheel
	Wheel	rearWheel
}

enumeration Wheel {
	W14
	W16
	W18
	W20
}

behavior {
	require frontWheel = rearWheel
}


The specification says that a bike has a `frontWheel` and a `rearWheel`.

Both are of type `Wheel` and must have one of these four values: `W14`, `W16`, `W18`, or `W20`. Each value represents a different size.

Additionally, the value of the `frontWheel` must be equal to the value of the `rearWheel`, i.e., both wheels should be of the same size.

Examples of more complex bikes can be found in the directory `coom/instances`.

## From COOM to ASP facts

In the directory `coom/parser` you can find a translator from COOM to ASP facts.

The documentation about the fact format is available at [doc/fact-format.ipynb](doc/fact-format.ipynb).

To run the translator you need to install the library antlr4 for Python.



In [ ]:
! pip install antlr4-tools
! pip install antlr4-python3-runtime==4.9.3

We translate the previous basic bike as follows:

In [3]:
! python coom/parser/parse.py coom/instances/bike-1-basic.coom

structure("ROOT").
feature("ROOT","frontWheel","Wheel",1,1).
feature("ROOT","rearWheel","Wheel",1,1).

enumeration("Wheel").
option("Wheel", "W14").
option("Wheel", "W16").
option("Wheel", "W18").
option("Wheel", "W20").

behavior(("ROOT",0)).
require(("ROOT",0),"frontWheel=rearWheel").
binary("ROOT","frontWheel=rearWheel","frontWheel","=","rearWheel").
path("frontWheel",0,"frontWheel").
path("rearWheel",0,"rearWheel").



The first facts says that the `ROOT` structure has exactly `1` `frontWheel` of type `Wheel`.
> Note:
The numbers `1` and `1` represent the minimum and maximum number of front wheels or rear wheels per bike, respectively.
For simplicity, in this project, we can consider that those numbers are always `1` and `1`,
but *optionally* you can consider also other values.

The second group of facts describe the possible values of a `Wheel`, 
and the third group of facts describe the constraint over the wheels.

The translation of some examples can be found in `asp/instances`.


## Basic ASP Encoding

The file `asp/configuration.lp` contains an ASP encoding to solve a very basic configuration problem.

The instance for this encoding must be in the fact format described above. 

In this basic setting, we assume that the instance only uses the constructs of the basic bike example from above.

In [1]:
! cat asp/configuration.lp

% choose one option V for every feature F of (enumeration) type T
{ value(F,V) : option(T,V) } = 1 :- feature(_,F,T,_,_).

% it cannot be that the constraint (S,C) requires a binary formula B and B does not hold
:- behavior((S,C)), require((S,C),B), not holds_binary(B).

% binary formula B of the form Left = Right
holds_binary(B) :- binary(_,B,Left,"=",Right), value(Left,LV), value(Right,RV), LV = RV.

% display value
#show value/2.


We can run the encoding, together with the result of the translation of [coom/instances/bike-1-basic.coom](coom/instances/bike-1-basic.coom) that is available at [asp/instances/bike-1-basic.lp](asp/instances/bike-1-basic.lp).

In [3]:
! clingo asp/configuration.lp asp/instances/bike-1-basic.lp 0

clingo version 5.5.0
Reading from asp/configuration.lp ...
Solving...
Answer: 1
value("rearWheel","W18") value("frontWheel","W18")
Answer: 2
value("rearWheel","W20") value("frontWheel","W20")
Answer: 3
value("rearWheel","W16") value("frontWheel","W16")
Answer: 4
value("rearWheel","W14") value("frontWheel","W14")
SATISFIABLE

Models       : 4
Calls        : 1
Time         : 0.002s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.002s


There are four answers, one for each possible value of the wheels.

## Basic Clinguin Encoding

The file `asp/ui.lp` contains an ASP encoding to represent a very basic UI for product configuration using clinguin.

It has to be combined with an ASP encoding for the configuration problem and a given instance.

In [2]:
! cat asp/ui.lp

#const root_name="ROOT".

% main window
elem(w, window, root).
        
    % title
    elem(menu_bar, menu_bar, w). 
    attr(menu_bar, title, "Product configuration").

    % one container per feature
    elem(c(F), container, w) :- feature(root_name,F,T,_,_).
    attr(c(F), width, "500") :- feature(root_name,F,T,_,_).
    attr(c(F), class, ("m-3";"p-3";"bg-primary";"bg-opacity-50";"rounded"))           :- feature(root_name,F,T,_,_).
    attr(c(F), class, ("d-inline-flex";"flex-row-reverse";"justify-content-between")) :- feature(root_name,F,T,_,_).

        % label
        elem(l(F), label, c(F)) :- feature(root_name,F,T,_,_).
        attr(l(F), label, F) :- feature(root_name,F,T,_,_).

        % dropdown menu
        elem(ldd(F), dropdown_menu, c(F)) :- feature(root_name,F,T,_,_).
        attr(ldd(F), class, ("border-dark";"bg-primary";"bg-opacity-50")) :- feature(root_name,F,T,_,_).
        % select V for F if all answer sets have value V for F
        attr(ldd(F), selected, V) :- 

To install and understand clinguin, please read the documentation:
* https://clinguin.readthedocs.io

Once clinguin is installed, you can run the basic example using this command:
```
clinguin client-server --domain-files asp/configuration.lp asp/instances/bike-1-basic.lp --ui-files asp/ui.lp
```

This will generate a very basic UI that allows us to select the type of one of the wheels, and after that selection is done it automatically selects the same type for the other.

<img src="img/bike-1-basic-ui.png" width="600" align="center">


# Tasks

There are 3 tasks in this project:
1. Write an ASP encoding in `asp/configuration.lp` to represent the configuration problems of the instances at `asp/instances`. The encoding should also work for other instances that use the same elements of the COOM language.
2. Write an ASP encoding in `asp/ui.lp` for clinguin to generate a UI for configuration problems. This should be combined with an ASP encoding in `asp/configuration-clinguin.lp` that should be a small adaptation of `asp/configuration.lp` for clinguin.
3. Write some instance (or instances) in COOM to describe a new product configuration problem. The product can be anything that is of interest to you, except bikes. These new instances should make use of all the elements of the COOM language that are used in the given instances.

Task 1 can be checked automatically with the script `test.py` and the command:
```
python asp/test.py -e asp/configuration.lp -i asp/instances -s asp/solutions -t 100
```
This will check the 3 instances available at `asp/instances`. 

The directory `asp/instances` contains the translations of the 3 simplest instances from `coom/instances`. 
The other instance, `bike-4-city.coom`, uses more advanced features of COOM than the others. 
It is *optional* to extend the encoding to handle that instance and those features.


For task 2, the file `configuration-clinguin.lp` contains some hints on the adaptation for clinguin:

In [3]:
! cat asp/configuration-clinguin.lp

%*

 This file should be an adaptation to clingo of the file configuration.lp.

 Among other things, it can add new atoms in the heads of some of the integrity constraints
 to explain the incorrect choices made by the user.

 For example, the constraint 
   :- behavior((S,C)), require((S,C),B), not holds_binary(B).
 can become
   error((S,C),B) :- behavior((S,C)), require((S,C),B), not holds_binary(B).
 and the program ui.lp can refer to those error/2 atoms.

*% 


*Optionally*, to provide explanations, you can use the `ExplanationBackend` of clinguin, that does not require any modification of the configuration encoding. Click [here](https://github.com/potassco/clinguin/tree/master/examples/angular/sudoku_advanced) to see an example of its usage.

Tasks 2 and 3 should be run using a command of this form:
```
clinguin client-server --domain-files asp/configuration-clinguin.lp asp/instances/new-1.lp --ui-files asp/ui.lp
```
where `asp/instances/new-1.lp` should be the translation of some new COOM instance to ASP facts.

The UI should have at least two versions or options:
* In the first one, the user can only select those elements that are consistent with the previous selections.
* In the second one, the user can make any selection, and if they make a selection that is inconsistent with the previous selections,
  then the UI should provide an explanation for the inconsistency, 
  i.e., it should tell the user which constraints where violated by the selection.
  
As an *optional* extension of the project, the UI may allow the user to specify some preferences over the configurations.

# Framework for Task 1

The directory ``asp`` contains the files that you need for this task. In the directory ``asp/instances`` you can find the instances (our example is ``bike-1-basic.lp``) and in the directory ``asp/solutions`` you can find their solutions in ``json`` format. 

For this task you have to submit a file named ``configuration.lp``, included as a template in the directory ``asp``, that contains the following lines (and no more ``#show`` statements) so that in the output only occur the atoms of predicate ``value/2``:

```
#show value/2.
```

To run your encoding with our example, you can use this command:
* ``clingo asp/configuration.lp asp/instances/bike-1-basic.lp``

You can check if your encoding solves correctly all instances by running the ``Python`` script ``test.py`` as follows:
* ``python asp/test.py -e asp/configuration.lp -i asp/instances -s asp/solutions -t 100``

In this case, the timeout for each instance is set to `100` seconds, but you can use any other value instead.

For help, type `python asp/test.py --help`.

We recommend you to work locally in your computer, using your own installation of ``clingo``.

For this, you can run the next cell to generate a zip file of this directory. The zip file will be stored in the parent directory with the name `configuration.zip`. You can click on the folder symbol at the left of the screen to look for it and download it.

In [ ]:
import os
from shutil import make_archive
make_archive('../configuration', 'zip', os.getcwd())

You can also run your encoding in the next cell. It is not recommended to work in this notebook at ``Binder``, but if you do it, remember to download the files that you modify to your computer, otherwise you will lose your changes.

In [ ]:
%%clingo 0 asp/instances/bike-2-extended.lp -

% Modify this encoding please...

% choose one option V for every feature F of (enumeration) type T
{ value(F,V) : option(T,V) } = 1 :- feature(_,F,T,_,_).

% it cannot be that the constraint (S,C) requires a binary formula B and B does not hold
:- behavior((S,C)), require((S,C),B), not holds_binary(B).

% binary formula B of the form Left = Right
holds_binary(B) :- binary(_,B,Left,"=",Right), value(Left,LV), value(Right,RV), LV = RV.

% display value
#show value/2.

# Formalities
You can work on the solution alone or in groups of two people. 
Different groups have to submit different solutions, in case
of plagiarism all groups involved will fail the project. 

Your solution should represent correctly all solutions for every instance.
In this project, the instances are small and should be solved fast. 
Hence, they all should be solved within the time limit.
This is tested automatically by the script ``test.py``. 

You will have to present your UI in class. 
We will send you further instructions about this from Moodle.

# Tips
 
* If you are stuck you can contact us. We will do out best to answer all your questions. You can send us questions and remarks either via Moodle or by email.

* Start as soon as possible to avoid running out of time. However, if you still realize that you have problems making it before the deadline, please contact us instead of copying another solution.